## ThirdAI's NeuralDB

NeuralDB, as the name suggests, is a combination of a neural network and a database. It provides a high-level API for users to insert different types of files into it and search through the file contents with natural language queries. The neural network part of it enables semantic search while the database part of it stores the paragraphs of the files that are inserted into it.

First, let's install the dependencies.

In [9]:
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/c9/b7/f511d1ddfaa1acc04356a7f75ca389bf9e6e5bd19ac368bc786de4063240/langchain-0.0.252-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.11 from https://files.pythonhosted.org/packages/bb/63/5016b578465eeee84c630bb38439b1d5c9aca9ccc312843178fdfdc9a5b4/langsmith-0.0.18-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.4 MB 1.9 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.4 MB 4.4 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.4 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.198
    Uninstalling langchain-0.0.198:
      Successfully uninstalled langchain-0.0.198


In [1]:
from thirdai import licensing, neural_db as ndb
licensing.deactivate()
licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")

Now, let's import the relevant module and define a neural db class.

In [2]:
db = ndb.NeuralDB(user_id="my_user") # you can use any username, in the future, this username will let you push models to the model hub

### You even load from a base DB from our Bazaar (optional but recommended)

We have a model bazaar that provides users with domain specific NeuralDBs that can jumpstart searching on their private documents. The Bazaar has two main types of DBs

1. Base DBs: These come with models that have either general QnA capabilities or domain specific capabilities like search on Medical Documents, Financial documents or Contracts. These come with an empty data index into which users can insert their files.

2. Pre-Indexed DBs: These are ready-to-search DBs that come with pre-trained models and their corresponding datasets. These are meant to  search through large public datasets like PubMed or Amazon 3MM Products or Stackoverflow issues etc.

In [13]:
# Set up a cache directory
import os
if not os.path.isdir("bazaar_cache"):
    os.mkdir("bazaar_cache")

from pathlib import Path
from model_bazaar import Bazaar
bazaar = Bazaar(cache_dir=Path("bazaar_cache"))


Call fetch to refresh list of available DBs.

In [14]:
bazaar.fetch() # Optional arg filter="model name" to filter by model name.


Below is the list of all DBs in the Bazaar.

In [15]:
print(bazaar.list_model_names())


['Contract Review', 'Finance QnA', 'General QnA']


Finally load the DB

In [17]:
db = bazaar.get_model("General QnA")

Downloading: 100%|██████████| 820M/820M [02:05<00:00, 6.51MB/s] 


### Insert your files

Let's insert things into it!

Currently, we natively support adding CSV, PDF and DOCX files. We also have a support to automatically scrape and parse URLs. All other file formats have to be converted into CSV files where each row represents a paragraph/text-chunk of the document. 

#### Example 1: CSV files
The first example below shows how to insert a CSV file. Please note that a CSV file is required to have a column named "DOC_ID" with rows numbered from 0 to n_rows-1.

In [17]:
insertable_docs = []
csv_files = ['sample_nda.csv']

for file in csv_files:
    csv_doc = ndb.CSV(
        path=file,
        id_column="DOC_ID",
        strong_columns=["passage"],
        weak_columns=["para"],  
        reference_columns=["passage"])
    #
    insertable_docs.append(csv_doc)


#### Example 2: PDF files

In [3]:
import nltk
nltk.download('punkt')

insertable_docs = []
pdf_files = ['sample_nda.pdf']

for file in pdf_files:
    pdf_doc = ndb.PDF(file)
    insertable_docs.append(pdf_doc)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Advay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Example 3: DOCX files

In [9]:
insertable_docs = []
doc_files = ['sample_nda.docx']

for file in doc_files:
    doc = ndb.DOCX(file)
    insertable_docs.append(doc)

#### Example 4: Parse from URLs directly

In [ ]:
valid_url_data = ndb.parsing_utils.recursive_url_scrape(base_url="https://www.thirdai.com/pocketllm/", max_crawl_depth=0)

In [11]:
insertable_docs = []

for url, response in valid_url_data:
    try:
        insertable_docs.append(ndb.URL(url, response))
    except:
        continue

### Insert into NeuralDB

If you wish to insert without unsupervised training, you can set 'train=False' in the insert() method.

In [4]:
source_ids = db.insert(insertable_docs, train=False)

The above command is intended to be used with a base DB which already has reasonable knowledge of the domain. In general, we always recommend using 'train=True' as shown below.

#### Insert and Train

In [5]:
source_ids = db.insert(insertable_docs, train=True)

If you call the insert() method multiple times, the documents will automatically be de-duplicated. If insert=True, then the training will be done multiple times.

### Search

Now let's start searching.

In [6]:
search_results = db.search(
    query="what is the termination period",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

In consideration of the business discussions disclosure of Confidential Information and any future business relationship between the parties it is hereby agreed as follows: 1. CONFIDENTIAL INFORMATION. For purposes of this Agreement the term "Confidential Information" shall mean any information business plan concept idea know-how process technique program design formula algorithm or work-in-process Request for Proposal (RFP) or Request for Information (RFI) and any responses thereto engineering manufacturing marketing technical financial data or sales information or information regarding suppliers customers employees investors or business operations and other information or materials whether disclosed in written graphic oral or electronic form whether tangible or intangible and in whatever form or medium provided or which is learned or disclosed in the course of discussions studies or other work undertaken between the parties prior to or after the Effective Date.
************
12. ENTIR

We can see that the search pulled up the right passage that contains the termination period "(i) five (5) years or (ii) when the confidential information no longer qualifies as a trade secret" .

In [20]:
search_results = db.search(
    query="made by and between",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).
************
in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023
************


We can see that the search pulled up the right passage again that has "made by and between".

Now let's ask a tricky question.

In [21]:
search_results = db.search(
    query="who are the parties involved?",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text)
    # print(result.context(radius=1))
    # print(result.source)
    # print(result.metadata)
    print('************')

6. excluded information. the parties agree that confidential information of the other party shall not include any information to the extent that the information: (i) is or at any time becomes a part of the public domain through no act or omission of the receiving party; (ii) is independently discovered or developed by the receiving party without use of the disclosing party’s confidential information; (iii) is rightfully obtained from a third party without any obligation of confidentiality; or (iv) is already known by the receiving party without any obligation of confidentiality prior to obtaining the confidential information from the disclosing party.
************
2. need to know. the receiving party shall limit its disclosure of the other party’s confidential information to those of its officers and employees and subcontractors (i) to which such disclosure is necessary for purposes of the discussions contemplated by this agreement and (ii) who have agreed in writing to be bound by pro

Oops! looks like when we search for "parties involved", we do not get the correct paragraph in the 1st position (we should be expecting the first paragraph as the correct results instead fo the last). 

No worries, we'll show shot to teach the model to correct it's retrieval.

### RLHF

Let's go over some of NeuralDB's advanced features. The first one is text-to-text association. This allows you to teach the model that two keywords, phrases, or concepts are related.

Based on the above example, let's teach the model that "parties involved" and the phrase "made by between" are the same.

In [22]:
db.associate(source="parties involved", target="made by and between")

Let's search again with the same query.

In [23]:
search_results = db.search(
    query="who are the parties involved?",
    top_k=2,
)

for result in search_results:
    print(result.text)
    # print(result.source)
    # print(result.metadata)
    print('************')

confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).
************
in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023
************


There you go! In just a line, you taught the model to correct itself and retrieve the correct result.

Now, let's see the 2nd option which is text-to-result association. Let's say that you know that "parties involved" should go the paragraph with DOC_ID=0, you can simply teach the model to associate the query to the corresponding label using the following API.

In [ ]:
db.text_to_result("made by and between",0)

If you want to use the above RLHF methods in a batch instead of a single sample, you can simply use the batched versions of the APIs as shown next.

In [ ]:
db.associate_batch([("parties involved","made by and between"),("date of signing","duly executed")])

In [ ]:
db.text_to_result_batch([("parties involved",0),("date of signing",16)])

### Supervised Training (Optional)

If you have supervised data for a specific CSV file in your list, you can simply train the DB on that file by specifying a source_id = source_ids[*file_number_in_your_list*].

Note: The supervised file should have the query_column and id_column that you specify in the following call. The id_column should match the id_column that you specified in the "Prep CSV Data" step or default to "DOC_ID".

In [ ]:
sup_files = ['sample_nda_sup.csv']

db.supervised_train([ndb.Sup(path, query_column="QUERY", id_column="DOC_ID", source_id=source_ids[0]) for path in sup_files])

### Get Answers from OpenAI using Langchain

In this section, we will show how to use LangChain and query OpenAI's QnA module to generate an answer from the references that you retrieve from the above DB. You'll have to specify your own OpenAI key for this module to work. You can replace this segment with any other generative model of your choice. You can choose to use an source model like MPT or Dolly for answer generation with the same prompt that you use with OpenAI.

In [24]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [25]:
from langchain.chat_models import ChatOpenAI
from paperqa.prompts import qa_prompt
from paperqa.chains import make_chain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo', 
    temperature=0.1,
)

qa_chain = make_chain(prompt=qa_prompt, llm=llm)

In [26]:
def get_references(query):
    search_results = db.search(query,top_k=3)
    references = []
    for result in search_results:
        references.append(result.text)
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context='\n\n'.join(references[:3]), answer_length="abt 50 words")

In [27]:
query = "what is the effective date of this agreement?"

references = get_references(query)
print(references)

['confidentiality agreement this confidentiality agreement (the “agreement”) is made by and between acme. dba tothemoon inc. with offices at 2025 guadalupe st. suite 260 austin tx 78705 and starwars dba tothemars with offices at the forest moon of endor and entered as of may 3 2023 (“effective date”).', 'in witness whereof this agreement has been duly executed by the parties hereto as of the latest date set forth below: acme inc. starwars inc. by: by: name: bugs bunny name: luke skywalker title: ceo title: ceo date: may 5 2023 date: may 7 2023', 'in consideration of the business discussions disclosure of confidential information and any future business relationship between the parties it is hereby agreed as follows: 1. confidential information. for purposes of this agreement the term “confidential information” shall mean any information business plan concept idea know-how process technique program design formula algorithm or work-in-process request for proposal (rfp) or request for inf

In [28]:
answer = get_answer(query, references)

print(answer)

The effective date of this agreement is May 3, 2023 (Context).


### Load and Save
As usual, saving and loading the DB are one-liners.

In [ ]:
# save your db
db.save("sample_nda.db")

# Loading is just like we showed above, with an optional progress handler
db.from_checkpoint("sample_nda.db", on_progress=lambda fraction: print(f"{fraction}% done with loading."))